# Add `forename` and `name` properties to `Person`

In [ ]:
import ast
import spacy
from spacytei.train import batch_train

## load trainings material

* annotations done by Martin Kirnbauer
* using prodigy

In [ ]:
file = r"C:\Users\pandorfer\Documents\Redmine\rita_project\vfbr\data\samples\samples_names.jsonl"

In [ ]:
with open(file) as f:
    TRAIN_DATA = f.readlines()

In [ ]:
train_data = [ast.literal_eval(x) for x in TRAIN_DATA]

## data transformation

* unfortunately prodigies native data output can't be used for training directly
* some transformation is needed

In [ ]:
def extract_ents(spans):
    ents = {"entities": []}
    for x in spans:
        ents["entities"].append((x['start'], x['end'], x['label']))
    return ents

In [ ]:
new_train_data = [((x['text']), extract_ents(x['spans'])) for x in train_data]
new_train_data[4]

## train a new model

* with label type `VN`

In [ ]:
batch_train(train_data=new_train_data, output_dir='./data/names_model', new_label='VN', eval_split=0.1, n_iter=10)

# annotate Persons

## forenames

In [ ]:
nlp = spacy.load(r"./data/names_model")

In [ ]:
no_vn = Person.objects.filter(forename="")
no_vn.count()

In [ ]:
for x in no_vn:
    doc = nlp("{}".format(x.written_name))
    for vn in doc.ents[:1]:
        if vn.label_ == "VN":
            x.forename = "{}".format(vn.text)
    x.save()

## surnames

In [ ]:
batch_train(train_data=new_train_data, output_dir='./data/names_model', new_label='NN', eval_split=0.1, n_iter=10)

In [ ]:
nlp = spacy.load(r"./data/names_model")

In [ ]:
no_nn = Person.objects.filter(name="")
no_nn.count()

In [ ]:
for x in no_nn:
    doc = nlp("{}".format(x.written_name))
    for ent in doc.ents[1:2]:
        if ent.label_ == "NN":
#             print(ent.text)
            x.name = "{}".format(ent.text)
    x.save()

## surnames part II
* try to get (even) better results with a new model, trained on 742 samples, validated against 82
* accuracy 0.96

In [ ]:
nlp = spacy.load(r"C:\Users\pandorfer\Documents\Redmine\prodigy\invs\invs_nn")